As part of your homework today, please fill out this form: https://forms.gle/QmhjRur2HKopBRLb6

We're going to be using the `statsmodels` library today, so please make sure you install it!

In [ ]:
!pip install statsmodels

# Regression vs Classification
* **Regression** is about predicting a continuous number (like what's the temeprature going to be tomorrow given the air pressure, wind speed, and the temperature today?).
* This is as opposed to **classification**, which is about predicting a label from a limited set of possible labels (like is it going to rain tomorrow? Yes or No?)


# Linear Regression
**Linear** regression is about finding a linear formula that best describes given data.

Let's start with one dimension, i.e. we will be predicting our value just based on one input.

Linear regression will help us find a straight line of best fit to these points to describe the relationship between $x$ and $y$.

Let's say we have a dataset that describes how long students prepared for an exam and what score they got in that exam.
![lin_reg1](https://sapiezynski.com/algoaudits/lr_01.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_02.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_03.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_04.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_05.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_06.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_07.png)

![lin_reg1](https://sapiezynski.com/algoaudits/lr_08.png)

What are the values of intercept and slope in our example?

And so, linear regression lets us describe the relationship between $y$ and $x$ following this linear formula:
$$ test\_score = \beta_0 + \beta_1time\_spent, $$
where $\beta_0$ is the intercept and $\beta_1$ is the slope.

Remember that the intercept ($\beta_0$) is the predicted value when $x=0$ and the slope ($\beta_1$) is the change in $y$ associated with a **unit change** in $x$.

We can use linear regression to describe relationships with more dimensions (variables) too:
$$ y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n $$
Here each $x_1, ..., x_n$ is a different dimension/variable, for example:
* the student's attendance
* their GPA
* the number of the month when they were born

Of course, not all variables will be predictive of the performance in test. Hopefully the coefficient associated with the month number will be close to 0, indicating that the month is not so important in predicting performance.

Let's load this data and and fit our linear regression:

In [ ]:
import pandas as pd
dataset = pd.read_csv('https://sapiezynski.com/algoaudits/lin_regression.csv')

dataset

When doing regression using the popular library `statsmodels` we will have to add a constant to find the intercept.


In [ ]:
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

dataset = add_constant(dataset)

In [ ]:
model = sm.OLS(dataset['test_score'], dataset[['const', 'hours_of_studying', 'month_of_birth']]).fit()
model.summary()

Linear regression might not be a very powerful model, but it offers a clear interpretation/explanation in suitable problems.

# Logistic Regression

**Logistic** regression is mostly used for binary classification (confusing!), predicting:
* "yes" vs "no",
* "success" vs "failure",
* "**is** on top of amazon search results" vs "**is not** on top of amazon search results".

Its formula looks similar to the linear regression, but now the outcome variable is the logarithm of odds of success instead of $y$.

$$ \ln(\frac{\pi}{1-\pi}) = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... $$

where $\pi$ is the probability of success, so $1-\pi$ is the probability of failure.

The values of all of the $\beta$s will come from `statsmodels` once we fit the model with the data.
The values of the different $x$s will be the values of our variables: `reviews_delta`, `stars delta`, `is_amazon_delta`, `is_ad_delta`.
So the right side of the equation will be a given, but what does the left side mean?

The left side is the log odd ratios, and a unit change in $x_1$ corresponds to $\beta_1$ change in log odd ratios.

Unfortunately, "log odds ratio" is not something we can easily interpret.

To be able to describe the model weights with intuitive language, let's rewrite this formula to calculate the probability of success $\pi$ from the values of $x$ directly:

$$\begin{align}
\ln(\frac{\pi}{1-\pi}) &= \beta_0 + \beta_1x_1 + \beta_2x_2 + ... \\
\end{align}
$$

Remeber that logarithm is the inverse function to exponentiation and $\ln$ is the logarithm with $e$ in the base, $\ln x=\log_e x$. To get rid of the logarithm we raise $e$ to the power of both sides of the equation:

$$\begin{align}
\frac{\pi}{1-\pi} &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
\end{align}$$

Then we multiply both sides by $(1-\pi)$, and then transform to get $\pi$:
$$\begin{align}
{\pi} &= ({1-\pi})e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
{\pi} &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}-\pi e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}\\
\end{align}$$

$$\begin{align}
{\pi}(1+e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}) &= e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...} \\
{\pi} &= \frac{e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}}{1+e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}}
\end{align}$$

So, while we cannot directly say from that a unit change of $x_1$ corresponds to a certain change in the probability of the outcome, we can just plug zeros and ones into the equation above and compute the changes.

In [ ]:
# downloading the dataset from Wednesday
pairs_dataset = pd.read_csv('pairs_dataset.csv')

pairs_dataset.dropna(inplace=True)
pairs_dataset

In [ ]:
# doing the train/test split as last time
from sklearn.model_selection import train_test_split


train, test = train_test_split(pairs_dataset, 
                               test_size=0.2, 
                               stratify=pairs_dataset['is_placed_higher'])

Let's start with a model that only has the intercept (constant) and `is_amazon_delta` as features.

In [ ]:
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant

# adding the intercept as a variable
train = add_constant(train)

# selecting which features to train on
features = ['const', 'is_amazon_delta']

# training a logistic regression model. First the dependent (outcome variable), then the features
log_reg = sm.Logit(train['is_placed_higher'], train[features]).fit()

In [ ]:
log_reg.summary()

We will mostly be looking at the bottom table.

As a reminder: this is the equation for probability in Logistic Regression:

$$ {\pi} = \frac{e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}}{1+e^{\beta_0 + \beta_1x_1 + \beta_2x_2 + ...}} $$

`const` (the intercept) translates to the probability $\pi$ (of product A being on top of the results list) if all other variables are equal to 0

$$ \pi = \frac{e^{\beta_{const}}}{1+e^{\beta_{const}}} $$

Let's calculate what that is:

In [ ]:
import numpy as np

const = log_reg.params['const']

np.e**const/(1+np.e**const) 

This should be very close to 50% (or 0.5). We look at the intercept when all the variables are zero. In our case there is only one variable, `is_amazon_delta`, which means that either:
1. Both products are from amazon brands, or
2. Neither of the products is from amazon brands.

We don't have a way to differentiate them, so it's a coin toss (50%)

The other coefficients describe the change in odds that corresponds to a unit change in the variable.
* if the coefficient is positive, it means that an **increase** of variable corresponds to an **increase** in the odds of the positive outcome
* if the coefficient is positive, it means that an **increase** of variable corresponds to an **decrease** in the odds of the positive outcome


So what happens when `is_amazon_delta` is equal to 1, i.e. product A is an amazon product and product B isn't?
By looking at the table with the regression summary we already know it's going to go up (because the coefficient is positive). But by how much? Let's calculate:


$$ \pi = \frac{e^{\beta_{const} + \beta_{is\_amazon\_delta}}}{1+e^{\beta_{const} + \beta_{is\_amazon\_delta}}} $$

In [ ]:
import numpy as np
numerator = np.e**(log_reg.params['const'] + log_reg.params['is_amazon_delta'])
print(numerator/(1+numerator))

That means that in this dataset 93\% of the time we have two products where one is from amazon and the other isn't, it's the amazon product that will be on the first place.

Is our investigation solved? Well no, because maybe this is just because amazon products have higher ratings or more reviews. Let's include the rating difference in the analysis to **control** for this:

In [ ]:
features = ['const', 'is_amazon_delta','rating_delta']
log_reg = sm.Logit(train['is_placed_higher'], train[features]).fit()
log_reg.summary()

Now notice the last two columns - that's the 95% confidence interval. The model estimates that the "real" value of the coefficient is between these two numbers. If one value is positive, and the other negative, the interval includes 0, which means we can't really tell if the effect is positive, or negative. We say that **it's not significant**. 

Turns out, controling for the rating difference doesn't change anything and the difference between ratings of the two products do not have a statistically significant relationship with the order of products in this setup.

Let's add the review delta:

In [ ]:
features = ['const', 'is_amazon_delta','rating_delta', 'review_count_delta']
log_reg = sm.Logit(train['is_placed_higher'], train[features]).fit()
log_reg.summary()

Same story, let's also include the ad delta:

In [ ]:
features = ['const', 'is_amazon_delta','rating_delta', 'review_count_delta', 'is_ad_delta']
log_reg = sm.Logit(train['is_placed_higher'], train[features]).fit()
log_reg.summary()

The only other significant coefficient is `is_ad_delta`. How do we interpret its value?

Let's take two products, such that:
* neither of them are from amazon, so `is_amazon_delta`=0
* they have the same rating, so `is_rating_delta`=0
* they have the same review count, so `review_count_delta`=0
* product A is an ad and product B is not, so `is_ad_delta`=0

Now we can plug these values into the equation:

In [ ]:
numerator = np.e**(log_reg.params['const']\
                   + log_reg.params['is_amazon_delta'] * 0\
                   + log_reg.params['rating_delta'] * 0\
                   + log_reg.params['review_count_delta'] * 0\
                   + log_reg.params['is_ad_delta'] * 1)

print(numerator/(1+numerator))

That's the probability the ad will be displayed first over an equivalent product that is not an ad.

Let's take another example.
Products A and B have the same mean rating and are not ads.
Product A is from amazon brands. 
Product B is not, but it has 500,000 reviews more. What is the probability that A is displayed first?


In [ ]:
numerator = np.e**(log_reg.params['const']\
                   + log_reg.params['is_amazon_delta'] * 1\
                   + log_reg.params['rating_delta'] * 0\
                   + log_reg.params['review_count_delta'] * (-500000)\
                   + log_reg.params['is_ad_delta'] * 0)

print(numerator/(1+numerator))

500,000 more reviews are not enough to beat the Amazon advantage! What about a 800,000?

In [ ]:
numerator = np.e**(log_reg.params['const']\
                   + log_reg.params['is_amazon_delta'] * 1\
                   + log_reg.params['rating_delta'] * 0\
                   + log_reg.params['review_count_delta'] * (-800000)\
                   + log_reg.params['is_ad_delta'] * 0)

print(numerator/(1+numerator))

There we go, if a non-amazon product has more than 800,000 reviews more than a similar amazon product, it might be displayed higher. 


## Actually using logistic regression in practice.
All of this math was to show you that there is a real meaning behind these numbers. It would of course be very impractical if we had to manually calculate this every time, so we can just use the model to compute these probabilities:

In [ ]:
log_reg.predict([[1, 1, 0, -800000, 0]]) # be careful with the feature order here!

You might also recall that I said logistic regression is usually used for binary classification, rather than predicting a continuous number, like we've been doing so far.

In practice, it means that we apply a threshold at 0.5:
* If the output of logistic regression is higher than 0.5, we say it's a positive prediction (in our case product A is displayed on top).
* If the output is lower than 0.5, we say it's a negative prediction  (in our case product B is displayed on top).


# Logistic Regression vs RandomForests
We've now shown that Logistic Regression gave us similar results to RandomForest from last class: that a product being from amazon is the most predictive variable of whether it comes on top. Being an ad is also important though less and the difference in the count of reviews barely matters in the face of the other two variables.

It also let us quantify how much exactly these variables matter.

But leaving interpretability aside, does it do as well as RandomForests at actually predicting which product comes on top? Let's compare:

In [ ]:
test = add_constant(test)
y_pred = log_reg.predict(test[features])

In [ ]:
((y_pred > 0.5) == test['is_placed_higher']).mean()

Compared to the RandomForests on the same train/test split:


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, # how many trees?
                             max_depth = 3, # the deeper the tree the more accuracy in training but more potential for overfitting
                             max_features = 4) # the more features more accuracy in training but more potential for overfitting

clf.fit(train[features], train['is_placed_higher'])
y_pred_rf = clf.predict(test[features])
(y_pred_rf == test['is_placed_higher']).mean()

As we know from last class, this number is feeble and changes as we change the train/test split, but the point is that LogisticRegression is necessarily more interpretable, and sometimes this doesn't even come at a cost of predictive perfomance.

In [ ]:
data = pd.read_csv('https://www.sapiezynski.com/cs4910/markup/data.csv')

In [ ]:
data.columns

In [ ]:
features = ['search_term', 'placed_higher',\
            'stars_delta', 'reviews_delta','is_shipped_by_amazon',\
            'is_sold_by_amazon','is_amazon','is_top_clicked']

In [ ]:
for key in ['is_shipped_by_amazon',\
            'is_sold_by_amazon','is_amazon','is_top_clicked']:
    data[key] /= 2

In [ ]:
data[features].to_csv('the_markup.csv', index=False)

# Exercise 1
1. Load the data originally published by The Markup:
```python
markup_df = pd.read_csv('https://www.sapiezynski.com/algoaudits/the_markup.csv') 
```
2. Make sure to add the constant so you can compute the intercept.
2. Decide which features (columns) you will use in the analysis - use all that make sense to include.
4. Split the data into train and test sets
5. Train a Logistic Regression model on the training set and inspect the coefficients.
6. Answer the following questions:
    1. Which variables have statistically significant coefficients?
    2. Increases in which variables correspond to higher probability of product A coming on top?
    3. What's the probability that product A will be displayed above B if:
       * B is an amazon brand product and A is not
       * B has 1,000,000 more reviews than A
       * A is top clicked and B is not
       * other variables are the same
    3. Perform the same calculation but now B has only 1,000 more reviews than A. Is this what you expected? Why yes/no?
    4. All other things equal, what's the probability that the amazon product comes on top if the other product is not from amazon?
7. Don't forget to fill out the form! https://forms.gle/QmhjRur2HKopBRLb6

In [ ]:
markup_df = pd.read_csv('https://www.sapiezynski.com/algoaudits/the_markup.csv')

In [ ]:
markup_df